In [ ]:
#Code is performing text clustering and sentiment analysis on a dataset of product listing.
#pandas for data manipualtion
#Tfidfvectorizer to convert text to numarical features using TF-IDf
#kMeans to perform k-mean clustering
#silhoutte_score to evaluate the clusteringquality.
#sentimentIntensityAnalyzer to perform sentiment analysis


import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

nltk.download('stopwords') #to ensure stopwords are doenloaded

# Load NLTK's sentiment analyzer
sid = SentimentIntensityAnalyzer()

data = pd.read_csv('Product listing.csv')

# Data preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Tokenization can be done using regex or libraries like NLTK or spaCy
    # Here, a simple split by space is used
    tokens = text.split()
    # Remove stopwords (you may need to download the stopwords list for your language)
    stopwords = set(['the', 'and', 'is', 'in', 'to', 'it', 'this', 'of', 'for', 'with', 'as'])
    tokens = [token for token in tokens if token not in stopwords]
    return ' '.join(tokens)

data['clean_text'] = data['product'].apply(preprocess_text)

# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
tfidf_matrix = tfidf_vectorizer.fit_transform(data['clean_text'])

# Clustering with K-means
k = 5  # Number of clusters (you can adjust this)
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(tfidf_matrix)

# Assign cluster labels to each review
data['cluster_label'] = kmeans.labels_

# Sentiment Analysis
def get_sentiment(text):
    # NLTK's sentiment analyzer
    sentiment_scores = sid.polarity_scores(text)
    # Classify sentiment based on compound score
    if sentiment_scores['compound'] >= 0.05:
        return 'Positive'
    elif sentiment_scores['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'
    
data['sentiment'] = data['clean_text'].apply(get_sentiment)


# Evaluate clustering using silhouette score
silhouette_avg = silhouette_score(tfidf_matrix, kmeans.labels_)
print(f"Silhouette Score: {silhouette_avg}")

# Display some reviews from each cluster
for cluster_id in range(k):
    cluster_samples = data[data['cluster_label'] == cluster_id].sample(5)  # Displaying 5 samples per cluster
    print(f"\nCluster {cluster_id}:")
    for index, row in cluster_samples.iterrows():
        print(row['product'])
        print("Sentiment:", row['sentiment'])
        print('-' * 50)
        
# Visualize clusters using PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(tfidf_matrix.toarray())
data['pca1'] = principal_components[:, 0]
data['pca2'] = principal_components[:, 1]

plt.figure(figsize=(10, 7))
for cluster in range(k):
    clustered_data = data[data['cluster_label'] == cluster]
    plt.scatter(clustered_data['pca1'], clustered_data['pca2'], label=f'Cluster {cluster}')
plt.legend()
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('K-means Clusters Visualization')
plt.show()

